In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
class LayeredHeisenbergMC:
    def __init__(self, L=10, Lz=10, p_weak=0.0):
        self.L = L
        self.Lz = Lz
        self.N = L * L * Lz
        
        # Initialize Spins: Random 3D unit vectors
        # Shape: (L, L, Lz, 3)
        spins = np.random.randn(L, L, Lz, 3)
        norms = np.linalg.norm(spins, axis=3, keepdims=True)
        self.spins = spins / norms
        
        # --- LAYERED DISORDER SETUP (Eq. 2 in Paper) ---
        # J_para depends only on Z layer index
        # Strong Layer (J=1.0), Weak Layer (J=0.2)
        # p_weak is the concentration of weak layers (p in the paper)
        self.J_para = np.ones(Lz) 
        self.J_perp = np.ones(Lz) # Inter-layer coupling usually constant or correlated
        
        if p_weak > 0:
            # Randomly assign layers as "Weak"
            is_weak = np.random.rand(Lz) < p_weak
            self.J_para[is_weak] = 0.2  # Weak intra-layer coupling
            # The paper mentions J_perp can also be random, 
            # but usually J_para is the dominant effect for layered physics.
            
    def energy(self):
        """Calculate Total Energy (Hamiltonian Eq. 1)"""
        E = 0
        
        # 1. Intra-layer interactions (neighbors in x and y)
        # We vectorise this: S(r) dot S(r+x)
        Sx = np.sum(self.spins * np.roll(self.spins, 1, axis=0), axis=3)
        Sy = np.sum(self.spins * np.roll(self.spins, 1, axis=1), axis=3)
        
        # Multiply by J_para which varies by Z layer
        # J_para shape (Lz,) needs broadcasting to (L, L, Lz)
        J_broadcast = self.J_para[np.newaxis, np.newaxis, :]
        E -= np.sum(J_broadcast * (Sx + Sy))
        
        # 2. Inter-layer interactions (neighbors in z)
        Sz = np.sum(self.spins * np.roll(self.spins, 1, axis=2), axis=3)
        J_perp_broadcast = self.J_perp[np.newaxis, np.newaxis, :]
        E -= np.sum(J_perp_broadcast * Sz)
        
        return E

    def mc_step(self, T, delta=0.2):
        """Metropolis Step for Heisenberg Spins"""
        # We cannot just flip spins; we rotate them randomly.
        # Pick random sites
        for _ in range(self.N):
            x, y, z = np.random.randint(0, self.L), np.random.randint(0, self.L), np.random.randint(0, self.Lz)
            
            old_spin = self.spins[x, y, z].copy()
            
            # Propose new spin: add small random vector and re-normalize
            perturbation = np.random.uniform(-delta, delta, 3)
            new_spin = old_spin + perturbation
            new_spin /= np.linalg.norm(new_spin)
            
            # Calculate local energy change
            # Neighbors
            nbrs = (
                self.spins[(x+1)%self.L, y, z] + self.spins[(x-1)%self.L, y, z] +
                self.spins[x, (y+1)%self.L, z] + self.spins[x, (y-1)%self.L, z]
            )
            # Z-neighbors need specific J_perp handling
            # (Simplified for clarity: assuming J_perp constant for local update calculation)
            nbr_z_up = self.spins[x, y, (z+1)%self.Lz]
            nbr_z_down = self.spins[x, y, (z-1)%self.Lz]
            
            # Local H_eff
            # Note: Explicit index checking needed for varying J
            J_local = self.J_para[z]
            dE_interaction = -J_local * np.dot(new_spin - old_spin, nbrs)
            dE_z = -1.0 * np.dot(new_spin - old_spin, nbr_z_up + nbr_z_down) # Assuming J_perp=1
            
            dE = dE_interaction + dE_z

            if dE < 0 or np.random.rand() < np.exp(-dE / T):
                self.spins[x, y, z] = new_spin
                
    def simulate(self, T, steps=500):
        # Equilibration
        for _ in range(200):
            self.mc_step(T)
            
        data = []
        for _ in range(steps):
            self.mc_step(T)
            data.append(self.spins.copy())
        return np.array(data)

# --- Data Generation Strategy ---
# 1. Pure System (p_weak=0.0) -> Label 0 (Ordered) / Label 1 (Disordered)
# 2. Layered System (p_weak=0.5) -> Test Data to find Griffiths Phase

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Input shape: (L, L, Lz, 3)
# 3 channels correspond to Sx, Sy, Sz
model = models.Sequential([
    layers.Input(shape=(10, 10, 10, 3)),
    
    # 3D Convolution to catch spatial correlations in all directions
    layers.Conv3D(32, (3, 3, 3), activation='relu'),
    layers.MaxPooling3D((2, 2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax') # [P(Ordered), P(Disordered)]
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [3]:
# --- 3. EXECUTION: Generate Data and Train ---

def generate_dataset(n_samples=20):
    """
    Generates a balanced dataset of Ordered (Low T) vs Disordered (High T)
    n_samples: Total samples (half ordered, half disordered)
    """
    X = []
    y = []
    
    print(f"Generating {n_samples} samples...")
    
    # Half Ordered (Low Temp, T=0.1)
    print("  > Simulating Ordered Phase (T=0.1)...")
    for i in range(n_samples // 2):
        # Initialize system (p_weak=0 for pure system training)
        sim = LayeredHeisenbergMC(L=10, Lz=10, p_weak=0.0)
        
        # Run simulation (reduced steps for speed during testing)
        # We take the FINAL state of the simulation as one data point
        traj = sim.simulate(T=0.1, steps=100) 
        final_state = traj[-1]  # Shape (10, 10, 10, 3)
        
        X.append(final_state)
        y.append(0) # Label 0 = Ordered

    # Half Disordered (High Temp, T=10.0)
    print("  > Simulating Disordered Phase (T=10.0)...")
    for i in range(n_samples // 2):
        sim = LayeredHeisenbergMC(L=10, Lz=10, p_weak=0.0)
        
        traj = sim.simulate(T=10.0, steps=100)
        final_state = traj[-1]
        
        X.append(final_state)
        y.append(1) # Label 1 = Disordered

    return np.array(X), np.array(y)

# 1. Generate the data
X_train, y_train = generate_dataset(n_samples=20) # Small batch for testing

# 2. Check shapes
print(f"\nData Shape: {X_train.shape}") # Should be (20, 10, 10, 10, 3)
print(f"Labels Shape: {y_train.shape}")

# 3. Train the Model
print("\nStarting Training...")
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=4,
    validation_split=0.2
)

print("\nTraining Complete! TensorBoard/Graphs can be plotted now.")

Generating 20 samples...
  > Simulating Ordered Phase (T=0.1)...
  > Simulating Disordered Phase (T=10.0)...

Data Shape: (20, 10, 10, 10, 3)
Labels Shape: (20,)

Starting Training...
Epoch 1/5
4/4 [==============================] - 3s 364ms/step - loss: 0.5609 - accuracy: 0.6250 - val_loss: 0.0424 - val_accuracy: 1.0000
Epoch 2/5
4/4 [==============================] - 0s 19ms/step - loss: 0.2294 - accuracy: 0.8750 - val_loss: 0.4426 - val_accuracy: 1.0000
Epoch 3/5
4/4 [==============================] - 0s 20ms/step - loss: 0.1383 - accuracy: 0.9375 - val_loss: 0.0505 - val_accuracy: 1.0000
Epoch 4/5
4/4 [==============================] - 0s 18ms/step - loss: 0.0443 - accuracy: 1.0000 - val_loss: 0.0240 - val_accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 0s 17ms/step - loss: 0.0317 - accuracy: 1.0000 - val_loss: 0.0314 - val_accuracy: 1.0000

Training Complete! TensorBoard/Graphs can be plotted now.


In [5]:
import sys
print("Notebook is using this Python:")
print(sys.executable)

Notebook is using this Python:
c:\Users\PARAM\Documents\Gemini-Test\DDP\boson_env\Scripts\python.exe


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

def generate_large_dataset(n_samples=10000, L=10):
    """
    Generates synthetic 3D Heisenberg spin configurations for 4 classes.
    L: Linear size of the lattice (L x L x L)
    n_samples: Total number of samples to generate
    """
    X = []
    Y = []
    
    samples_per_class = n_samples // 4
    print(f"Generating {n_samples} samples ({samples_per_class} per class)...")
    
    # --- CLASS 0: STRONGLY DISORDERED (SD) ---
    # Physics: High Temp (T > Tu). Pure Paramagnetic.
    # Visual: Completely random 3D vectors.
    print("Generating Class 0: Strongly Disordered...")
    for _ in range(samples_per_class):
        # Generate random 3D vectors
        config = np.random.randn(L, L, L, 3)
        # Normalize to unit length (Heisenberg spins)
        config /= np.linalg.norm(config, axis=3, keepdims=True)
        X.append(config)
        Y.append(0)

    # --- CLASS 1: WEAKLY DISORDERED (WD) - GRIFFITHS PHASE ---
    # Physics: Tu > T > Tc. Bulk is Disordered, but Rare Strong Slabs are Ordered.
    # Visual: Random noise + A thick "slab" of aligned spins.
    print("Generating Class 1: Weakly Disordered (Griffiths)...")
    for _ in range(samples_per_class):
        config = np.random.randn(L, L, L, 3)
        config /= np.linalg.norm(config, axis=3, keepdims=True)
        
        # Create a "Rare Strong Slab" (Thickness 3 to 5 layers)
        slab_thickness = np.random.randint(3, 6)
        z_start = np.random.randint(0, L - slab_thickness)
        
        # Pick a random direction for the slab's order
        direction = np.random.randn(3)
        direction /= np.linalg.norm(direction)
        
        # Apply order to the slab (with small thermal noise)
        slab_noise = np.random.normal(0, 0.2, (L, L, slab_thickness, 3))
        config[:, :, z_start:z_start+slab_thickness, :] = direction + slab_noise
        
        # Re-normalize the slab part
        # (We only normalize the slice we modified to keep it physical)
        slice_norm = np.linalg.norm(config[:, :, z_start:z_start+slab_thickness, :], axis=3, keepdims=True)
        config[:, :, z_start:z_start+slab_thickness, :] /= slice_norm
        
        X.append(config)
        Y.append(1)

    # --- CLASS 2: WEAKLY ORDERED (WO) - GRIFFITHS PHASE ---
    # Physics: Tc > T > Tl. Bulk is Ordered, but Rare Weak Slabs are Disordered.
    # Visual: Aligned spins + A thick "slab" of random noise.
    print("Generating Class 2: Weakly Ordered (Griffiths)...")
    for _ in range(samples_per_class):
        # Start with global order
        direction = np.random.randn(3)
        direction /= np.linalg.norm(direction)
        
        # Create base config with thermal noise
        config = np.ones((L, L, L, 3)) * direction
        config += np.random.normal(0, 0.3, (L, L, L, 3))
        config /= np.linalg.norm(config, axis=3, keepdims=True)
        
        # Create a "Rare Weak Slab" (Disordered Region)
        slab_thickness = np.random.randint(3, 6)
        z_start = np.random.randint(0, L - slab_thickness)
        
        # Generate noise for the slab
        noise_slab = np.random.randn(L, L, slab_thickness, 3)
        noise_slab /= np.linalg.norm(noise_slab, axis=3, keepdims=True)
        
        # Insert the disordered slab
        config[:, :, z_start:z_start+slab_thickness, :] = noise_slab
        
        X.append(config)
        Y.append(2)

    # --- CLASS 3: STRONGLY ORDERED (SO) ---
    # Physics: T < Tl. Pure Ferromagnetic.
    # Visual: Almost perfectly aligned spins (very low noise).
    print("Generating Class 3: Strongly Ordered...")
    for _ in range(samples_per_class):
        direction = np.random.randn(3)
        direction /= np.linalg.norm(direction)
        
        config = np.ones((L, L, L, 3)) * direction
        # Very low thermal noise
        config += np.random.normal(0, 0.1, (L, L, L, 3))
        config /= np.linalg.norm(config, axis=3, keepdims=True)
        
        X.append(config)
        Y.append(3)

    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.int32)

In [2]:
# 1. Generate Data
L_SIZE = 10
N_SAMPLES = 10000  # Large dataset

X, Y = generate_large_dataset(n_samples=N_SAMPLES, L=L_SIZE)

# Split into Train and Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Training Data Shape: {X_train.shape}")
print(f"Testing Data Shape: {X_test.shape}")

# 2. Define the 3D CNN Model
model = models.Sequential([
    layers.Input(shape=(L_SIZE, L_SIZE, L_SIZE, 3)),
    
    # First Conv Block
    layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    
    # Second Conv Block (Added for better feature extraction on larger data)
    layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    # Note: Depending on L_SIZE, a second MaxPooling might reduce dimensions too much.
    # For L=10, MaxPool (2,2,2) -> (5,5,5). Another MaxPool would be too small.
    # So we skip the second pooling or use stride 1.
    
    layers.Flatten(),
    
    # Dense Layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3), # Dropout to prevent overfitting on synthetic patterns
    layers.Dense(4, activation='softmax') # 4 Output Neurons
])

# 3. Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. Train
history = model.fit(
    X_train, Y_train,
    epochs=15,            # Increased epochs for larger data
    batch_size=64,        # Batch processing
    validation_data=(X_test, Y_test)
)

# 5. Save the Model
model.save('heisenberg_griffiths_detector.keras')
print("Model trained and saved.")

Generating 10000 samples (2500 per class)...
Generating Class 0: Strongly Disordered...
Generating Class 1: Weakly Disordered (Griffiths)...
Generating Class 2: Weakly Ordered (Griffiths)...
Generating Class 3: Strongly Ordered...
Training Data Shape: (8000, 10, 10, 10, 3)
Testing Data Shape: (2000, 10, 10, 10, 3)
Epoch 1/15
125/125 [==============================] - 44s 344ms/step - loss: 0.2165 - accuracy: 0.9168 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 2/15
125/125 [==============================] - 35s 282ms/step - loss: 0.0037 - accuracy: 0.9994 - val_loss: 1.4535e-04 - val_accuracy: 1.0000
Epoch 3/15
125/125 [==============================] - 36s 289ms/step - loss: 5.5460e-04 - accuracy: 1.0000 - val_loss: 7.8340e-05 - val_accuracy: 1.0000
Epoch 4/15
125/125 [==============================] - 42s 336ms/step - loss: 4.3568e-04 - accuracy: 1.0000 - val_loss: 1.6636e-04 - val_accuracy: 1.0000
Epoch 5/15
125/125 [==============================] - 42s 333ms/step - loss: 4.8851e

In [4]:
Y_train

array([3, 0, 0, ..., 2, 0, 2])

In [9]:
def detect_phases(model, monte_carlo_data):
    """
    monte_carlo_data: Shape (N_temperature_steps, L, L, L, 3)
    Returns: Probabilities for [SD, WD, WO, SO] at each temperature
    """
    predictions = model.predict(monte_carlo_data)
    return predictions

# Example Usage (Conceptual):
# 1. Run MC Simulation across temperatures T = 4.0 down to 0.1
n_samples = 2000
t_lis = np.linspace(4.0, 0.1, num=20) # 20 temperature points
X = []
Y = []  
for t in t_lis:
    for i in range(n_samples // 2):
            sim = LayeredHeisenbergMC(L=10, Lz=10, p_weak=0.0)
            
            traj = sim.simulate(T=t, steps=100)
            final_state = traj[-1]
            
            X.append(final_state)
            if t > 3.0: 
                Y.append(0) # Label 1 = Disordered
            elif t > 1.5:
                Y.append(1) # Label 2 = Weakly Disordered
            elif t > 0.5:
                Y.append(2) # Label 3 = Weakly Ordered
            else:
                Y.append(3) # Label 4 = Strongly Ordered
            

# 2. Get predictions
probs = detect_phases(model, np.array(X))
err = np.abs(probs.argmax(axis=1) - np.array(Y))
print(f"Overall Accuracy: {(err == 0).mean() * 100:.2f}%")
# 3. Plotting the Griffiths Phase
# Plot probs[:, 1] (Weakly Disordered) vs Temperature.
# You should see a peak or a plateau in the Griffiths region (Tc < T < Tu).
plt.plot(t_lis, probs[:, 1], label='Weakly Disordered')
plt.plot(t_lis, probs[:, 0], label='Strongly Disordered')
plt.xlabel("Temperature")
plt.ylabel("Probability of Weak Phases")
plt.title("Griffiths Phase")
plt.legend()
plt.show()

KeyboardInterrupt: 